<a href="https://colab.research.google.com/github/rajeevfromkrec/pytorch/blob/master/RNN_LSTM_Pytorcvh_Many_to_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as tt

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=tt.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=tt.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
for images,labels in train_loader:
  print (images[0].shape)
  break

torch.Size([1, 28, 28])


In [ ]:
#RNN network many to one

In [19]:
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_classes):
    super().__init__()
    self.hidden_size=hidden_size
    self.num_layers=num_layers
    self.lstm=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
    self.fc=nn.Linear(hidden_size,num_classes)

  def forward(self,x):
    h0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    c0=torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    out,_ =self.lstm(x,(h0,c0))
    out=self.fc(out[:,-1,:])
    return out

In [20]:
model =RNN(input_size,hidden_size,num_layers,num_classes).to(device)

In [27]:
model

RNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [24]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [25]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Epoch [1/2], Step [100/600], Loss: 0.7091
Epoch [1/2], Step [200/600], Loss: 0.3834
Epoch [1/2], Step [300/600], Loss: 0.2088
Epoch [1/2], Step [400/600], Loss: 0.2285
Epoch [1/2], Step [500/600], Loss: 0.0297
Epoch [1/2], Step [600/600], Loss: 0.1661
Epoch [2/2], Step [100/600], Loss: 0.0770
Epoch [2/2], Step [200/600], Loss: 0.0590
Epoch [2/2], Step [300/600], Loss: 0.0765
Epoch [2/2], Step [400/600], Loss: 0.0740
Epoch [2/2], Step [500/600], Loss: 0.0406
Epoch [2/2], Step [600/600], Loss: 0.0735
Test Accuracy of the model on the 10000 test images: 97.48 %


In [28]:
torch.save(model.state_dict(),'model.ckpt')

In [29]:
!ls

model.ckpt  sample_data


In [30]:
!ls -lrt

total 848
drwxr-xr-x 1 root root   4096 Jul 30 16:30 sample_data
-rw-r--r-- 1 root root 860493 Aug 16 17:42 model.ckpt
